# Analyze competition

In [1]:
import json
import os

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters


register_matplotlib_converters()

%matplotlib inline

plt.style.use("seaborn")

In [2]:
data_directory = os.path.join("..", "..", "..", "data")

In [3]:
campaigns_file_path = os.path.join(data_directory, "suppression_campaigns.json")
if not os.path.exists(campaigns_file_path):
    with tarfile.open(os.path.join(data_directory, "suppression_campaigns.tar.xz"), "r", encoding="utf-8") as compressed_file:
        compressed_file.extract("suppression_campaigns.json", data_directory)

In [4]:
with open(campaigns_file_path, "r", encoding="utf-8") as campaigns_file:
    campaigns = []
    line = campaigns_file.readline().strip()
    while line != "":
        campaigns.append(json.loads(line))
        line = campaigns_file.readline().strip()

In [5]:
with open(os.path.join(data_directory, "suppression_bot_hash_to_cluster_id.json"), "r") as json_file:
    bot_hash_to_cluster_id = json.load(json_file)

In [6]:
attacks_by_victim = {}

for campaign in campaigns:
    bot = campaign["bot_address"]
    victim = campaign["suppressed_contract_address"]
    
    if victim not in attacks_by_victim:
        attacks_by_victim[victim] = []
    
    for round_ in campaign["rounds"]:
        first_block = round_["first_block"]
        number_of_blocks = round_["nr_of_blocks"]

        attacks_by_victim[victim].append({
            "bot_cluster_id": bot_hash_to_cluster_id[bot],
            "bot": bot,
            "first_block": round_["first_block"],
            "number_of_blocks": round_["nr_of_blocks"],
        })

In [7]:
competition_by_victim = {}

for victim in attacks_by_victim.keys():
    if len(attacks_by_victim[victim]) > 1:
        competition_by_victim[victim] = sorted(attacks_by_victim[victim], key=lambda attack: attack["first_block"])

In [8]:
for victim, attacks in competition_by_victim.items():
    attack = attacks[0]
    for next_attack in attacks[1:]:
        if attack["first_block"] + attack["number_of_blocks"] >= next_attack["first_block"]:
            print("victim:", victim)
            print("first attack:", attack)
            print("second attack:", next_attack)
            print("---")
        attack = next_attack

victim: 0xA62142888ABa8370742bE823c1782D17A0389Da1
first attack: {'bot_cluster_id': 4, 'bot': '0xd037763925C23f5Ba592A8b2F4910D051a57A9e3', 'first_block': 6316400, 'number_of_blocks': 2}
second attack: {'bot_cluster_id': 4, 'bot': '0xf9109191e455981dEEf5bAC0618Dcb0Aa86FAE7D', 'first_block': 6316402, 'number_of_blocks': 2}
---
victim: 0xA62142888ABa8370742bE823c1782D17A0389Da1
first attack: {'bot_cluster_id': 4, 'bot': '0xd037763925C23f5Ba592A8b2F4910D051a57A9e3', 'first_block': 6316405, 'number_of_blocks': 2}
second attack: {'bot_cluster_id': 4, 'bot': '0xf9109191e455981dEEf5bAC0618Dcb0Aa86FAE7D', 'first_block': 6316407, 'number_of_blocks': 1}
---
victim: 0xDd9fd6b6F8f7ea932997992bbE67EabB3e316f3C
first attack: {'bot_cluster_id': 4, 'bot': '0xd037763925C23f5Ba592A8b2F4910D051a57A9e3', 'first_block': 6232120, 'number_of_blocks': 2}
second attack: {'bot_cluster_id': 2, 'bot': '0xD6e8a9a1873fa97a75e0b8bc954aa5C698820273', 'first_block': 6232122, 'number_of_blocks': 4}
---
